In [1]:
#!pip install wolframalpha
import datetime as dt
import tqdm
import os
import pandas as pd
import glob
import numpy as np
import sys

import wolframalpha
app_id = 'JU4PR2-E8JGQVLHXW'
client = wolframalpha.Client(app_id)

In [2]:
from platform import python_version
print(python_version())

3.8.5


#small example
query = 'Precipitation amount in Zollikofen on October 6, 2020 1h'
res = client.query(query)
for t in res.results:
    print(t.text)

#small example
query = 'Precipitation amount in Zollikofen '+d.strftime('%B')+' '+str(d.day)+', '+str(d.year)+' '+str(d.hour)+'h'
res = client.query(query)
for t in res.results:
    print(t.text)

#small example
d = dt.datetime(2020,1,24,22,44,11)
query = 'Moon Phase calculator'+d.strftime('%B')+' '+str(d.day)+', '+str(d.year)+' '+str(d.hour)+'h'
res = client.query(query)
r = ''
for t in res.results:
    print(t.text.strip('|').strip(' '))
#for verification
#https://www.pleine-lune.org/calendrier-lunaire-2020
#res

In [3]:
li_type = ['Precipitation amount', 'temperature', 'humidity', 'wind speed','Moon Phase calculator'] #pressure
#path_ = r'D:\vm_exchange\PhD_Data\Weather'
path_ = r'C:\Users\camil\Desktop\vm_exchange\PhD_Data\Weather'

In [4]:
mi = dt.datetime(2020,9,29)
#ma = dt.datetime(2019,9,2)
ma = dt.datetime(2021,8,2)
df = pd.DataFrame(columns=['date', 'temperature', 'humidity', 'wind speed', 'Precipitation amount',
                           'Moon Phase calculator','query']) #pressure
path_weather = os.path.join(path_,'weather.csv')
if len(glob.glob(path_weather))==1:
    df = pd.read_csv(path_weather, sep=';', parse_dates=['date'])
    #remove the last date, in case we stop in middle of the data to grab
    #df = df[df['date']!=max(df['date'].unique())]
    df = df.drop_duplicates(subset=None, keep='first', inplace=False)
    print(df.shape)
    print('We already have %d dates of weather'%df.shape[0])
    #mi = max(mi, max(df['date'].tolist())) #not a good idea in case of errors as in that case we would like to try it again 
print(df[df.isin(["ERROR"]).any(axis=1)].shape)
display(df[df.isin(["ERROR"]).any(axis=1)])
df = df[~df.isin(["ERROR"]).any(axis=1)]
#remove the rows with at least one error
Daterange = pd.date_range(start=mi, end=ma, freq='H')   
print(len(Daterange))
Daterange = [d for d in Daterange if d not in df['date'].unique()]
print(len(Daterange))
if (len(Daterange)*len(li_type))>=2000:
    print('WARNING: to many calls to be completed these queries!')
print('You need %d new queries to complete this date interval'%(len(Daterange)*len(li_type)))
df = df.sort_values(['date'])
print(df.shape)
df.tail(5)

(5239, 7)
We already have 5239 dates of weather
(36, 7)


,date,temperature,humidity,wind speed,Precipitation amount,query,Moon Phase calculator
5158,2020-11-05 22:00:00,"8 °C\n(10:00 pm CET | Thursday, November 5, 2020)",ERROR,"1 m/s\n(10:00 pm CET | Thursday, November 5, 2...","(unknown)\n(10:00 pm CET | Thursday, November ...","Moon Phase calculatorNovember 5, 2020 22h",| waning gibbous moon\n76.55% illuminated
5159,2020-11-16 19:00:00,"7 °C\n(7:00 pm CET | Monday, November 16, 2020)",ERROR,"1 m/s\n(7:00 pm CET | Monday, November 16, 2020)","(unknown)\n(7:00 pm CET | Monday, November 16,...","Moon Phase calculatorNovember 16, 2020 19h",| waxing crescent moon\n3.48% illuminated
5160,2020-11-16 21:00:00,"5 °C\n(9:00 pm CET | Monday, November 16, 2020)",ERROR,"0 m/s\n(9:00 pm CET | Monday, November 16, 2020)","(unknown)\n(9:00 pm CET | Monday, November 16,...","Moon Phase calculatorNovember 16, 2020 21h",| waxing crescent moon\n3.86% illuminated
5161,2020-12-15 21:00:00,"4 °C\n(9:00 pm CET | Tuesday, December 15, 2020)",ERROR,"1 m/s\n(9:00 pm CET | Tuesday, December 15, 2020)","(unknown)\n(9:00 pm CET | Tuesday, December 15...","Moon Phase calculatorDecember 15, 2020 21h",| waxing crescent moon\n1.86% illuminated
5162,2021-02-08 17:00:00,"5 °C\n(5:00 pm CET | Monday, February 8, 2021)",ERROR,"5 m/s\n(5:00 pm CET | Monday, February 8, 2021)","(unknown)\n(5:00 pm CET | Monday, February 8, ...","Moon Phase calculatorFebruary 8, 2021 17h",| waning crescent moon\n11.27% illuminated
5163,2021-02-08 18:00:00,"4 °C\n(6:00 pm CET | Monday, February 8, 2021)",ERROR,"5 m/s\n(6:00 pm CET | Monday, February 8, 2021)","(unknown)\n(6:00 pm CET | Monday, February 8, ...","Moon Phase calculatorFebruary 8, 2021 18h",| waning crescent moon\n10.98% illuminated
5164,2021-02-08 19:00:00,"3 °C\n(7:00 pm CET | Monday, February 8, 2021)",ERROR,"4 m/s\n(7:00 pm CET | Monday, February 8, 2021)","(unknown)\n(7:00 pm CET | Monday, February 8, ...","Moon Phase calculatorFebruary 8, 2021 19h",| waning crescent moon\n10.69% illuminated
5165,2021-02-08 20:00:00,"2 °C\n(8:00 pm CET | Monday, February 8, 2021)",ERROR,"4 m/s\n(8:00 pm CET | Monday, February 8, 2021)","(unknown)\n(8:00 pm CET | Monday, February 8, ...","Moon Phase calculatorFebruary 8, 2021 20h",| waning crescent moon\n10.41% illuminated
5166,2021-02-09 09:00:00,"0 °C\n(9:00 am CET | Tuesday, February 9, 2021)",ERROR,"1 m/s\n(9:00 am CET | Tuesday, February 9, 2021)","(unknown)\n(9:00 am CET | Tuesday, February 9,...","Moon Phase calculatorFebruary 9, 2021 9h",| waning crescent moon\n7.06% illuminated
5167,2021-02-09 10:00:00,"0 °C\n(10:00 am CET | Tuesday, February 9, 2021)",ERROR,"1 m/s\n(10:00 am CET | Tuesday, February 9, 2021)","(unknown)\n(10:00 am CET | Tuesday, February 9...","Moon Phase calculatorFebruary 9, 2021 10h",| waning crescent moon\n6.82% illuminated


7369
2166
You need 10830 new queries to complete this date interval
(5203, 7)


,date,temperature,humidity,wind speed,Precipitation amount,query,Moon Phase calculator
5234,2021-05-05 02:00:00,"12 °C\n(2:00 am CEST | Wednesday, May 5, 2021)","average: 58% (58%)\n(2:00 am CEST | Wednesday,...","5 m/s\n(2:00 am CEST | Wednesday, May 5, 2021)","no precipitation\n(2:00 am CEST | Wednesday, M...","Moon Phase calculatorMay 5, 2021 2h",| waning crescent moon\n37.96% illuminated
5235,2021-05-05 03:00:00,"11 °C\n(3:00 am CEST | Wednesday, May 5, 2021)",average: 64% (62 to 67%)\n(3:00 am CEST | Wedn...,"4 m/s\n(3:00 am CEST | Wednesday, May 5, 2021)","rain\n(3:00 am CEST | Wednesday, May 5, 2021)","Moon Phase calculatorMay 5, 2021 3h",| waning crescent moon\n37.55% illuminated
5236,2021-05-05 04:00:00,"10 °C\n(4:00 am CEST | Wednesday, May 5, 2021)","average: 71% (71%)\n(4:00 am CEST | Wednesday,...","6 m/s\n(4:00 am CEST | Wednesday, May 5, 2021)","no precipitation\n(4:00 am CEST | Wednesday, M...","Moon Phase calculatorMay 5, 2021 4h",| waning crescent moon\n37.13% illuminated
5237,2021-05-05 05:00:00,"10 °C\n(5:00 am CEST | Wednesday, May 5, 2021)",average: 64% (62 to 66%)\n(5:00 am CEST | Wedn...,"6 m/s\n(5:00 am CEST | Wednesday, May 5, 2021)","no precipitation\n(5:00 am CEST | Wednesday, M...","Moon Phase calculatorMay 5, 2021 5h",| waning crescent moon\n36.72% illuminated
5238,2021-05-05 06:00:00,"9 °C\n(6:00 am CEST | Wednesday, May 5, 2021)",average: 69% (66 to 71%)\n(6:00 am CEST | Wedn...,"5 m/s\n(6:00 am CEST | Wednesday, May 5, 2021)","rain\n(6:00 am CEST | Wednesday, May 5, 2021)","Moon Phase calculatorMay 5, 2021 6h",| waning crescent moon\n36.31% illuminated


In [5]:
#not same as : https://weather.plus/observations-weather-station.php?dest=Bern-Zollikofen&id=066310&type=synop&day=20200713, our
#results are less consistent :( 
#ATTENTION: monthly api calls limited to 2'000
#if not working, save query and results: "ERROR", but continue the other query from the same date
#open if exist
li_df = []
for i,d in enumerate(tqdm.tqdm(Daterange)):
    now_ = dt.datetime.now()
    #dont get the data in advance
    if d<now_:      
        dico = {'date':d}
        for type_ in li_type:
            try:                    
                query = '%s in Zollikofen '%type_+d.strftime('%B')+' '+str(d.day)+', '+str(d.year)+' '+str(d.hour)+'h'
                if type_=='Moon Phase calculator':
                    query = type_+d.strftime('%B')+' '+str(d.day)+', '+str(d.year)+' '+str(d.hour)+'h'                    
                print('QUERY: ',query)
                dico['query'] = query
                res = client.query(query)
                for t in res.results:
                    if 'no data available' in t.text:
                        print('ERROR: no data available------------- ', query)
                        error
                    dico[type_] = t.text
            except Exception as e:
                print('ERROR:-------------', e, ' ---------- ', query)
                dico[type_] = 'ERROR'
        li_df.append(dico)
        dico = {}
        if i%10==0:
            df_ = pd.DataFrame(li_df)
            df = pd.concat([df,df_])
            df.to_csv(path_weather, index=False, sep=';')
            li_df = []
            print(df.shape)
            display(df.tail(2))
df_ = pd.DataFrame(li_df)
df = pd.concat([df,df_])
df.to_csv(path_weather, index=False, sep=';')
print(df.shape)
display(df.tail(2))

  0%|                                                                                         | 0/2166 [00:00<?, ?it/s]

QUERY:  Precipitation amount in Zollikofen November 5, 2020 22h
QUERY:  temperature in Zollikofen November 5, 2020 22h
QUERY:  humidity in Zollikofen November 5, 2020 22h
ERROR: no data available-------------  humidity in Zollikofen November 5, 2020 22h
ERROR:------------- name 'error' is not defined  ----------  humidity in Zollikofen November 5, 2020 22h
QUERY:  wind speed in Zollikofen November 5, 2020 22h
QUERY:  Moon Phase calculatorNovember 5, 2020 22h
(5204, 7)


,date,temperature,humidity,wind speed,Precipitation amount,query,Moon Phase calculator
5238,2021-05-05 06:00:00,"9 °C\n(6:00 am CEST | Wednesday, May 5, 2021)",average: 69% (66 to 71%)\n(6:00 am CEST | Wedn...,"5 m/s\n(6:00 am CEST | Wednesday, May 5, 2021)","rain\n(6:00 am CEST | Wednesday, May 5, 2021)","Moon Phase calculatorMay 5, 2021 6h",| waning crescent moon\n36.31% illuminated
0,2020-11-05 22:00:00,"8 °C\n(10:00 pm CET | Thursday, November 5, 2020)",ERROR,"1 m/s\n(10:00 pm CET | Thursday, November 5, 2...","(unknown)\n(10:00 pm CET | Thursday, November ...","Moon Phase calculatorNovember 5, 2020 22h",| waning gibbous moon\n76.55% illuminated


  0%|                                                                              | 1/2166 [00:22<13:24:44, 22.30s/it]

QUERY:  Precipitation amount in Zollikofen November 16, 2020 19h
QUERY:  temperature in Zollikofen November 16, 2020 19h
QUERY:  humidity in Zollikofen November 16, 2020 19h
ERROR: no data available-------------  humidity in Zollikofen November 16, 2020 19h
ERROR:------------- name 'error' is not defined  ----------  humidity in Zollikofen November 16, 2020 19h
QUERY:  wind speed in Zollikofen November 16, 2020 19h
QUERY:  Moon Phase calculatorNovember 16, 2020 19h


...

 12%|█████████                                                                 | 264/2166 [1:32:42<11:06:22, 21.02s/it]

QUERY:  Precipitation amount in Zollikofen May 14, 2021 19h
QUERY:  temperature in Zollikofen May 14, 2021 19h
ERROR: no data available-------------  temperature in Zollikofen May 14, 2021 19h
ERROR:------------- name 'error' is not defined  ----------  temperature in Zollikofen May 14, 2021 19h
QUERY:  humidity in Zollikofen May 14, 2021 19h
ERROR: no data available-------------  humidity in Zollikofen May 14, 2021 19h
ERROR:------------- name 'error' is not defined  ----------  humidity in Zollikofen May 14, 2021 19h
QUERY:  wind speed in Zollikofen May 14, 2021 19h
ERROR: no data available-------------  wind speed in Zollikofen May 14, 2021 19h
ERROR:------------- name 'error' is not defined  ----------  wind speed in Zollikofen May 14, 2021 19h
QUERY:  Moon Phase calculatorMay 14, 2021 19h


100%|████████████████████████████████████████████████████████████████████████████| 2166/2166 [1:33:02<00:00,  2.58s/it]

(5468, 7)


,date,temperature,humidity,wind speed,Precipitation amount,query,Moon Phase calculator
2,2021-05-14 18:00:00,"8 °C\n(6:00 pm CEST | Friday, May 14, 2021)","average: 100% (100%)\n(6:00 pm CEST | Friday, ...","1 m/s\n(6:00 pm CEST | Friday, May 14, 2021)","rain\n(6:00 pm CEST | Friday, May 14, 2021)","Moon Phase calculatorMay 14, 2021 18h",| waxing crescent moon\n7.28% illuminated
3,2021-05-14 19:00:00,ERROR,ERROR,ERROR,"(unknown)\n(7:00 pm CEST | Friday, May 14, 2021)","Moon Phase calculatorMay 14, 2021 19h",| waxing crescent moon\n7.49% illuminated


In [6]:
print(df[df.isin(["ERROR"]).any(axis=1)].shape)
df[df.isin(["ERROR"]).any(axis=1)]

(39, 7)


,date,temperature,humidity,wind speed,Precipitation amount,query,Moon Phase calculator
0,2020-11-05 22:00:00,"8 °C\n(10:00 pm CET | Thursday, November 5, 2020)",ERROR,"1 m/s\n(10:00 pm CET | Thursday, November 5, 2...","(unknown)\n(10:00 pm CET | Thursday, November ...","Moon Phase calculatorNovember 5, 2020 22h",| waning gibbous moon\n76.55% illuminated
0,2020-11-16 19:00:00,"7 °C\n(7:00 pm CET | Monday, November 16, 2020)",ERROR,"1 m/s\n(7:00 pm CET | Monday, November 16, 2020)","(unknown)\n(7:00 pm CET | Monday, November 16,...","Moon Phase calculatorNovember 16, 2020 19h",| waxing crescent moon\n3.48% illuminated
1,2020-11-16 21:00:00,"5 °C\n(9:00 pm CET | Monday, November 16, 2020)",ERROR,"0 m/s\n(9:00 pm CET | Monday, November 16, 2020)","(unknown)\n(9:00 pm CET | Monday, November 16,...","Moon Phase calculatorNovember 16, 2020 21h",| waxing crescent moon\n3.86% illuminated
2,2020-12-15 21:00:00,"4 °C\n(9:00 pm CET | Tuesday, December 15, 2020)",ERROR,"1 m/s\n(9:00 pm CET | Tuesday, December 15, 2020)","(unknown)\n(9:00 pm CET | Tuesday, December 15...","Moon Phase calculatorDecember 15, 2020 21h",| waxing crescent moon\n1.86% illuminated
3,2021-02-08 17:00:00,"5 °C\n(5:00 pm CET | Monday, February 8, 2021)",ERROR,"5 m/s\n(5:00 pm CET | Monday, February 8, 2021)","(unknown)\n(5:00 pm CET | Monday, February 8, ...","Moon Phase calculatorFebruary 8, 2021 17h",| waning crescent moon\n11.27% illuminated
4,2021-02-08 18:00:00,"4 °C\n(6:00 pm CET | Monday, February 8, 2021)",ERROR,"5 m/s\n(6:00 pm CET | Monday, February 8, 2021)","(unknown)\n(6:00 pm CET | Monday, February 8, ...","Moon Phase calculatorFebruary 8, 2021 18h",| waning crescent moon\n10.98% illuminated
5,2021-02-08 19:00:00,"3 °C\n(7:00 pm CET | Monday, February 8, 2021)",ERROR,"4 m/s\n(7:00 pm CET | Monday, February 8, 2021)","(unknown)\n(7:00 pm CET | Monday, February 8, ...","Moon Phase calculatorFebruary 8, 2021 19h",| waning crescent moon\n10.69% illuminated
6,2021-02-08 20:00:00,"2 °C\n(8:00 pm CET | Monday, February 8, 2021)",ERROR,"4 m/s\n(8:00 pm CET | Monday, February 8, 2021)","(unknown)\n(8:00 pm CET | Monday, February 8, ...","Moon Phase calculatorFebruary 8, 2021 20h",| waning crescent moon\n10.41% illuminated
7,2021-02-09 09:00:00,"0 °C\n(9:00 am CET | Tuesday, February 9, 2021)",ERROR,"1 m/s\n(9:00 am CET | Tuesday, February 9, 2021)","(unknown)\n(9:00 am CET | Tuesday, February 9,...","Moon Phase calculatorFebruary 9, 2021 9h",| waning crescent moon\n7.06% illuminated
8,2021-02-09 10:00:00,"0 °C\n(10:00 am CET | Tuesday, February 9, 2021)",ERROR,"1 m/s\n(10:00 am CET | Tuesday, February 9, 2021)","(unknown)\n(10:00 am CET | Tuesday, February 9...","Moon Phase calculatorFebruary 9, 2021 10h",| waning crescent moon\n6.82% illuminated


df['day'] = df['date'].map(lambda x: dt.datetime.date(x))
df['day'].value_counts()

#add lune to weather data
#dico_d_m = {}
for d in tqdm.tqdm(Daterange):
    now_ = dt.datetime.now()
    #dont get the data in advance
    if d<now_:   
        if d not in dico_d_m:
            try:
                query = 'Moon Phase calculator'+d.strftime('%B')+' '+str(d.day)+', '+str(d.year)+' '+str(d.hour)+'h'
                res = client.query(query) 
                for t in res.results:
                    dico_d_m[d] = t.text.strip('|').strip(' ')
            except Exception as e:
                print('ERROR')
                print(e)
#df['Moon Phase calculator'] = df['date'].map(lambda x: dico_d_m.get(x, 'ERROR'))
#print(df[df['Moonphase']=='ERROR'].shape)
df

df = df.drop(['MoonName','Moon Phase calculator'],axis=1)

# Process

In [7]:
#add weather as environemntal factor
path_ = r'C:\Users\camil\Desktop\vm_exchange\PhD_Data\Weather'
path_weather = os.path.join(path_,'weather.csv')
df_weather = pd.read_csv(path_weather, sep=';', parse_dates=['date'])
df_weather = df_weather.drop_duplicates(subset=None, keep='first', inplace=False)
#remove excpetions
x = ', 1, 0,  , °, C, \n, (, 1, 2, :, 0, 0,  , a, m,  , C, E, T , | , F, r, i, d, a, y, ,,  , O, c, t, o, b, e, r,  , 3, 0, ,,  , 2, 0, 2, 0, ),'
df_weather['temperature'] = df_weather['temperature'].replace(x, x.replace(', ',''))
x = '3×10^0 m/s\n(1:00 pm CET | Sunday, January 10, 2021)'
df_weather['wind speed'] = df_weather['wind speed'].replace(x, 'ERROR')
#remove the rows with at least one error
df_weather = df_weather[~df_weather.isin(["ERROR"]).any(axis=1)]
#date
df_weather.rename(columns={'date':'Timestamp'}, inplace=True)
df_weather['date'] = df_weather['Timestamp'].map(lambda x: dt.datetime(x.year,x.month,x.day))
df_weather['hour'] = df_weather['Timestamp'].map(lambda x: x.hour)
#add usefull variable
df_weather['temperature_C'] = df_weather['temperature'].map(lambda x: int(x.split('\n')[0].split(' ')[0]))
df_weather['wind_speed_mpers'] = df_weather['wind speed'].map(lambda x: int(x.split('\n')[0].split(' ')[0]))
df_weather['humidity_avgpercent'] = df_weather['humidity'].map(lambda x: int(x.split('%')[0].split(' ')[-1].strip()))
df_weather['rain'] = df_weather['Precipitation amount'].map(lambda x: x.split('\n')[0])
df_weather['rain'] = df_weather['rain'].replace('rain | snow','snow | rain')
df_weather.to_csv(os.path.join(path_,'Weather_process.csv'), index=False, sep=';')
print(df_weather.shape)
df_weather.head(3)

(5428, 13)


,Timestamp,temperature,humidity,wind speed,Precipitation amount,query,Moon Phase calculator,date,hour,temperature_C,wind_speed_mpers,humidity_avgpercent,rain
0,2020-09-29 00:00:00,"10 °C\n(12:00 am CEST | Tuesday, September 29,...",average: 100% (100%)\n(12:00 am CEST | Tuesday...,"2 m/s\n(12:00 am CEST | Tuesday, September 29,...","rain\n(12:00 am CEST | Tuesday, September 29, ...","Moon Phase calculatorSeptember 29, 2020 0h",| waxing gibbous moon\n91.87% illuminated,2020-09-29,0,10,2,100,rain
1,2020-09-29 01:00:00,"10 °C\n(1:00 am CEST | Tuesday, September 29, ...","average: 100% (100%)\n(1:00 am CEST | Tuesday,...","3 m/s\n(1:00 am CEST | Tuesday, September 29, ...","rain\n(1:00 am CEST | Tuesday, September 29, 2...","Moon Phase calculatorSeptember 29, 2020 1h",| waxing gibbous moon\n92.09% illuminated,2020-09-29,1,10,3,100,rain
2,2020-09-29 02:00:00,"10 °C\n(2:00 am CEST | Tuesday, September 29, ...","average: 100% (100%)\n(2:00 am CEST | Tuesday,...","3 m/s\n(2:00 am CEST | Tuesday, September 29, ...","rain\n(2:00 am CEST | Tuesday, September 29, 2...","Moon Phase calculatorSeptember 29, 2020 2h",| waxing gibbous moon\n92.30% illuminated,2020-09-29,2,10,3,100,rain


In [8]:
df_weather['MoonName'] = df_weather['Moon Phase calculator'].map(lambda x: x.strip('|').strip(' ').split('\n')[0] if x!='ERROR' else 'ERROR')
df_weather['Illumination_percentage'] = df_weather['Moon Phase calculator'].map(lambda x: float(x.strip('|').strip(' ').split('\n')[1].replace(' illuminated','').strip('%').strip(' ')) if x!='ERROR' else 'ERROR')
#print(df['MoaonName'].value_counts())
#df['Illumination_percentage'].value_counts()

dico_CD = {'ERROR':'ERROR', 
           'first quarter moon':'LC', 
           'full moon':'Lchangement', 
           'last quarter moon':'DC',
           'new moon':'Lchangement', 
           'waning crescent moon':'DC', 
           'waning gibbous moon':'DC',
           'waxing crescent moon':'LC', 
           'waxing gibbous moon':'LC'}
#it seems that the order of the events are as follow:
#full moon, waning gibbous moon, last quarter moon, waning crescent moon, 
#new moon, waxing crescent moon, first quarter moon, waxing gibbous moon,
#full moon,...
#defintion: De la Nouvelle Lune (éclat minimum) à la Pleine Lune (éclat maximum) on dit que la Lune est Croissante car 
#sa partie éclairée grossit de jour en jour. De la Pleine Lune à la Nouvelle Lune, on dit que la Lune est Décroissante.
df_weather['croissant_decroissant'] = df_weather['MoonName'].map(lambda x: dico_CD[x])
#lucie ascendatne et descendante n'est pas indiqué ici.
df_weather.head(3)

,Timestamp,temperature,humidity,wind speed,Precipitation amount,query,Moon Phase calculator,date,hour,temperature_C,wind_speed_mpers,humidity_avgpercent,rain,MoonName,Illumination_percentage,croissant_decroissant
0,2020-09-29 00:00:00,"10 °C\n(12:00 am CEST | Tuesday, September 29,...",average: 100% (100%)\n(12:00 am CEST | Tuesday...,"2 m/s\n(12:00 am CEST | Tuesday, September 29,...","rain\n(12:00 am CEST | Tuesday, September 29, ...","Moon Phase calculatorSeptember 29, 2020 0h",| waxing gibbous moon\n91.87% illuminated,2020-09-29,0,10,2,100,rain,waxing gibbous moon,91.87,LC
1,2020-09-29 01:00:00,"10 °C\n(1:00 am CEST | Tuesday, September 29, ...","average: 100% (100%)\n(1:00 am CEST | Tuesday,...","3 m/s\n(1:00 am CEST | Tuesday, September 29, ...","rain\n(1:00 am CEST | Tuesday, September 29, 2...","Moon Phase calculatorSeptember 29, 2020 1h",| waxing gibbous moon\n92.09% illuminated,2020-09-29,1,10,3,100,rain,waxing gibbous moon,92.09,LC
2,2020-09-29 02:00:00,"10 °C\n(2:00 am CEST | Tuesday, September 29, ...","average: 100% (100%)\n(2:00 am CEST | Tuesday,...","3 m/s\n(2:00 am CEST | Tuesday, September 29, ...","rain\n(2:00 am CEST | Tuesday, September 29, 2...","Moon Phase calculatorSeptember 29, 2020 2h",| waxing gibbous moon\n92.30% illuminated,2020-09-29,2,10,3,100,rain,waxing gibbous moon,92.30,LC


In [9]:
df_weather['croissant_decroissant'].value_counts()

DC             2571
LC             2553
Lchangement     304
Name: croissant_decroissant, dtype: int64

In [10]:
#to get a general udnerstanding of the data
df_moon = df_weather.groupby(['MoonName'])['Illumination_percentage'].agg(lambda x: list(x)).reset_index()
df_moon['min_illumination'] = df_moon['Illumination_percentage'].map(lambda x: min(x) if x!='ERROR' else 'ERROR' )
df_moon['max_illumination'] = df_moon['Illumination_percentage'].map(lambda x: max(x) if x!='ERROR' else 'ERROR' )
df_moon

,MoonName,Illumination_percentage,min_illumination,max_illumination
0,first quarter moon,"[44.7, 45.14, 45.58, 46.02, 46.46, 46.9, 47.34...",44.69,55.24
1,full moon,"[99.74, 99.76, 99.78, 99.79, 99.81, 99.82, 99....",99.72,99.99
2,last quarter moon,"[55.07, 54.65, 54.23, 53.81, 53.39, 52.96, 52....",44.73,55.22
3,new moon,"[0.27, 0.23, 0.21, 0.18, 0.16, 0.15, 0.14, 0.1...",0.00,0.28
4,waning crescent moon,"[44.38, 43.95, 43.51, 43.08, 42.65, 42.21, 41....",0.29,44.68
5,waning gibbous moon,"[99.71, 99.68, 99.65, 99.61, 99.58, 99.54, 99....",55.34,99.72
6,waxing crescent moon,"[0.32, 0.37, 0.42, 0.48, 0.54, 0.61, 0.68, 0.7...",0.28,44.64
7,waxing gibbous moon,"[91.87, 92.09, 92.3, 92.51, 92.73, 92.93, 93.1...",55.34,99.71


In [11]:
df_weather['query'].iloc[0]

'Moon Phase calculatorSeptember 29, 2020 0h'

In [12]:
df_daily_weather = df_weather[df_weather['hour'].isin([10,11,12,13,14,15,16,17])].copy()
df_daily_weather = df_daily_weather.groupby(['date']).agg(
    temperature_C=pd.NamedAgg(column='temperature_C', aggfunc=lambda x: np.mean(x)),
    humidity_avgpercent=pd.NamedAgg(column='humidity_avgpercent', aggfunc=lambda x: np.mean(x)),
    had_rainorsnow=pd.NamedAgg(column='rain', aggfunc=lambda x: int(all([i=='no precipitation' for i in x]))),
    moon_name=pd.NamedAgg(column='MoonName', aggfunc=lambda x: list(x)[0]),
    Illumination_percentage=pd.NamedAgg(column='Illumination_percentage', aggfunc=lambda x: np.mean(x))).reset_index()
df_daily_weather.to_csv(os.path.join(path_,'df_daily_weather.csv'), index=False, sep=';')
print(df_daily_weather.shape)
df_daily_weather.head(3)

(227, 6)


,date,temperature_C,humidity_avgpercent,had_rainorsnow,moon_name,Illumination_percentage
0,2020-09-29,12.000,92.125,0,waxing gibbous moon,94.56625
1,2020-09-30,12.250,80.125,1,waxing gibbous moon,98.01625
2,2020-10-01,14.125,74.875,0,waxing gibbous moon,99.66250


In [13]:
df_daily_weather.groupby(['had_rainorsnow'])['humidity_avgpercent'].agg(lambda x: np.mean(x))

had_rainorsnow
0    81.691406
1    66.024869
Name: humidity_avgpercent, dtype: float64